In [10]:
!pip install PyPDF2

You are using pip version 9.0.3, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [11]:
import PyPDF2 as pypdf
import regex as re
import numpy as np
import csv
import pandas as pd

In [12]:
PDFfile = open("2018-2019-Kentucky-State-University-Catalogue.pdf", "rb")
pdfread = pypdf.PdfFileReader(PDFfile)
pdfread

In [13]:
y = pdfread.getPage(164)
y.extractText().replace('\n', '')

'156  Course Descriptions  AFRICAN-AMERICAN STUDIES  AAS 303/ART 303: African-American Art History. A course designed to reveal contributions and accomplishments of African-American artists and the influence of traditional early African art on art of the world. Credit: 3 semester hours.  AAS 314/HIS 314: African-Americans in the United States. The history of African-Americans in the United States, their role in the development of this country, and their struggle for rights, privileges, and protections of citizenship. Prerequisite(s): HIS 201 and HIS 202, or consent of instructor. Credit: 3 semester hours.  AAS 323/MUS 323: African-American Music. A study of the development of African-American music, including spirituals, blues, jazz, symphonic literature, and miscellaneous songs. Open to non-Music majors. Credit: 3 semester hours  AAS 325/MUS 325: Jazz History and Literature. A survey of jazz, including its origins, major performers, and composers. All idioms and styles are studied usi

In [14]:
regex = r'([A-Z\s?]+)?\s{2}([A-Z]{3,4})\s+(\d{3,4})\/?(.+?):?(.+?)\.'

In [28]:
def get_data(csv_name, pdf_name, page_range, pattern, dep_group, abbrev_group, c_group, t_group,
            year, grad_status):
    """Takes in name of PDF in local folder and extracts data in given PAGE_RANGE 
    using regex expression PATTERN and given group numbers. YEAR is the catalog year.
    GRAD_STATUS is either None (if unknown), 'G', 'UG', or a number indicating at
    which number is the course graduate."""
    
    PDFfile = open(pdf_name, "rb")
    pdfread = pypdf.PdfFileReader(PDFfile)
    
    csv_writer = csv.writer(open(csv_name, 'w'))
    csv_writer.writerow(["Department Name", "Course Catalogue Number", "Course Name",
     "Course Description", "Graduate/Undergraduate", "Format", "Lab", 
     "Academic Catalogue Year"])
    FS_keys = ["agri", "food", "animal"]
    
    look_ahead = pattern + r'(.+?)(?=' + pattern + r')'
    
    dep_names = {}
    #test = page_range[0] #DEBUGGING
    for i in np.arange(page_range[0], page_range[1]):
        page_text = pdfread.getPage(i).extractText().replace('\n', '')
        matches = re.findall(look_ahead, page_text)
        last = re.sub(look_ahead, '', page_text)
        last = re.search(pattern + r'(.+)', last)
        #print(test) #DEBUGGING
        #test += 1 #DEBUGGING
        matches = matches + [(last.group(dep_group), last.group(abbrev_group), 
                    last.group(c_group), last.group(t_group), last.group(5))]
        
        for match in matches:  
            #DEBUG: print(match)
            # Getting full department name
            if not dep_names.get(match[abbrev_group - 1]):
                dep_names[match[abbrev_group - 1]] = match[dep_group - 1]
            dep_name = dep_names.get(match[abbrev_group - 1])
            if dep_name != None:
                saved_dep_name = dep_name
                if len(dep_name) <2 :
                    dep_name = match[abbrev_group - 1] #tried this need to improve it 
            if dep_name == None:
                dep_name = saved_dep_name
            
                
            # Other groups
            course_num = match[abbrev_group - 1] + (match[c_group - 1][:3]) #added the abbrevation in front of number for those that don't have department name separate

            title = str(match[t_group - 1])
            desc = str(match[4]) # assuming description is group 5
            
            #Filtering
            for key in FS_keys:
                if (re.search(key, title, flags=re.IGNORECASE) or 
                    re.search(key, desc, flags=re.IGNORECASE)):
                    G_UG = grad_status
                    if grad_status:
                        if type(grad_status) == int:
                            G_UG = "UG"
                            if int(t.group(1)) >= grad_status:
                                G_UG = "G"
                    lab = False
                    if (re.search(r'Laboratory|Lab', title, flags=re.IGNORECASE) or 
                    re.search(r'Laboratory|Lab', desc, flags=re.IGNORECASE)):
                        lab = True
                    csv_writer.writerow([dep_name, course_num, title, desc, 
                                             G_UG, None, lab, year])
                                            # Leaving format blank for now
                        
    print(dep_names)
                
kentucky_pattern = r'([A-Z\s?]+)?\s{2}?([A-Z]{3,4})\s+(\d{3,4})\/?(.+?):?(.+?)\.'
#Partial Test set:
get_data('KentuckyStateU.csv', "2018-2019-Kentucky-State-University-Catalogue.pdf", [164, 253], kentucky_pattern, 
         1, 2, 3, 5, '2018-2019', None)

AttributeError: 'NoneType' object has no attribute 'group'